In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv("twcs.csv")

In [3]:
df.head()

,tweet_id,author_id,inbound,created_at,text,response_tweet_id,in_response_to_tweet_id
0,1,sprintcare,False,Tue Oct 31 22:10:47 +0000 2017,@115712 I understand. I would like to assist y...,2,3.0
1,2,115712,True,Tue Oct 31 22:11:45 +0000 2017,@sprintcare and how do you propose we do that,NaN,1.0
2,3,115712,True,Tue Oct 31 22:08:27 +0000 2017,@sprintcare I have sent several private messag...,1,4.0
3,4,sprintcare,False,Tue Oct 31 21:54:49 +0000 2017,@115712 Please send us a Private Message so th...,3,5.0
4,5,115712,True,Tue Oct 31 21:49:35 +0000 2017,@sprintcare I did.,4,6.0


## OneHotEncoding

Data Cleaning

1. lowercasing

In [4]:
df["text_data"]=df["text"].str.lower()

In [5]:
df["text_data"]

0        @115712 i understand. i would like to assist y...
1            @sprintcare and how do you propose we do that
2        @sprintcare i have sent several private messag...
3        @115712 please send us a private message so th...
4                                       @sprintcare i did.
                               ...                        
24401    @115714 y’all are trash! why do i have to pay ...
24402    @122830 something isn't right, please send us ...
24403                       @sprintcare just did...thanks.
24404    @sprintcare seriously, it's been two months si...
24405                                             @122831 
Name: text_data, Length: 24406, dtype: object

In [6]:
import re

In [7]:
def remove_html_tags(text):
    clean = re.compile('<.*?>')
    return re.sub(clean, '', text)

In [8]:
df['text_data'] = df['text_data'].astype(str)
df['text_without_html'] = df['text_data'].apply(remove_html_tags)

In [9]:
df['text_without_html']

0        @115712 i understand. i would like to assist y...
1            @sprintcare and how do you propose we do that
2        @sprintcare i have sent several private messag...
3        @115712 please send us a private message so th...
4                                       @sprintcare i did.
                               ...                        
24401    @115714 y’all are trash! why do i have to pay ...
24402    @122830 something isn't right, please send us ...
24403                       @sprintcare just did...thanks.
24404    @sprintcare seriously, it's been two months si...
24405                                             @122831 
Name: text_without_html, Length: 24406, dtype: object

Remove URL

In [10]:
def remove_url(text):
    if isinstance(text, str):
        pattern = re.compile(r'https?://\S+|www\.\S+')
        return pattern.sub("", text)
    else:
        return text

In [11]:
df["text_without_url"]=remove_url(df['text_without_html'])

In [12]:
df["text_without_url"]

0        @115712 i understand. i would like to assist y...
1            @sprintcare and how do you propose we do that
2        @sprintcare i have sent several private messag...
3        @115712 please send us a private message so th...
4                                       @sprintcare i did.
                               ...                        
24401    @115714 y’all are trash! why do i have to pay ...
24402    @122830 something isn't right, please send us ...
24403                       @sprintcare just did...thanks.
24404    @sprintcare seriously, it's been two months si...
24405                                             @122831 
Name: text_without_url, Length: 24406, dtype: object

In [13]:
import string

In [14]:
string.punctuation

'!"#$%&\'()*+,-./:;<=>?@[\\]^_`{|}~'

In [15]:
def remove_punc(text):
    translator = str.maketrans('', '', string.punctuation)
    return text.translate(translator)

In [16]:
df['without_punc'] = df['text_without_url'].apply(remove_punc)

In [17]:
df["without_punc"]

0        115712 i understand i would like to assist you...
1             sprintcare and how do you propose we do that
2        sprintcare i have sent several private message...
3        115712 please send us a private message so tha...
4                                         sprintcare i did
                               ...                        
24401    115714 y’all are trash why do i have to pay fo...
24402    122830 something isnt right please send us a d...
24403                            sprintcare just didthanks
24404    sprintcare seriously its been two months since...
24405                                              122831 
Name: without_punc, Length: 24406, dtype: object

In [22]:
def chat_conversion(text):
    new_text=[]
    for w in text.split():
        if w.upper() in chat_words:
            new_text.append(chat_words[w.upper()])
        else:
            new_text.append(w)
    return " ".join(new_text)

In [25]:
df["chat_conversion"]=df["without_punc"].str.split()

In [26]:
df["chat_conversion"]

0        [115712, i, understand, i, would, like, to, as...
1        [sprintcare, and, how, do, you, propose, we, d...
2        [sprintcare, i, have, sent, several, private, ...
3        [115712, please, send, us, a, private, message...
4                                     [sprintcare, i, did]
                               ...                        
24401    [115714, y’all, are, trash, why, do, i, have, ...
24402    [122830, something, isnt, right, please, send,...
24403                        [sprintcare, just, didthanks]
24404    [sprintcare, seriously, its, been, two, months...
24405                                             [122831]
Name: chat_conversion, Length: 24406, dtype: object

In [50]:
import textblob

In [51]:
from textblob import TextBlob

def spell_check(text):
    blob = TextBlob(text)
    return str(blob.correct())

In [52]:
df['spell_checked'] = df['chat_conversion'].apply(spell_check)

In [53]:
df['spell_checked']

0          The is a smile text with some misled words.
1          He are experiencing technical difficulties.
2    Spell check should help correct the misspelled...
Name: spell_checked, dtype: object

In [54]:
pip install nltk

In [55]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [56]:
from nltk.corpus import stopwords

In [57]:
def remove_stopwords(text):
  new_text=[]

  for word in text.split():
    if word in stopwords.words("english"):
      new_text.append("")
    else:
      new_text.append(word)


  return " ".join(new_text).replace(" ","")

In [58]:
df["stop_words"] = df["spell_checked"].apply(remove_stopwords)

In [59]:
df["stop_words"]

0                 Thesmiletextmisledwords.
1     Heexperiencingtechnicaldifficulties.
2    Spellcheckhelpcorrectmisspelledwords.
Name: stop_words, dtype: object

In [63]:
import emoji

In [64]:
def remove_emoji(text):
  clean_text=emoji.demojize(text)
  return clean_text

In [65]:
df["remove_emoji"] = df["stop_words"].apply(remove_emoji)

In [66]:
df["remove_emoji"]

0                 Thesmiletextmisledwords.
1     Heexperiencingtechnicaldifficulties.
2    Spellcheckhelpcorrectmisspelledwords.
Name: remove_emoji, dtype: object

In [67]:
import nltk
nltk.download('punkt')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [69]:
def tokenize_text(text):
    return nltk.word_tokenize(text)

In [70]:
df['tokens'] = df['remove_emoji'].apply(tokenize_text)

In [71]:
df['tokens']

0                 [Thesmiletextmisledwords, .]
1     [Heexperiencingtechnicaldifficulties, .]
2    [Spellcheckhelpcorrectmisspelledwords, .]
Name: tokens, dtype: object

In [72]:
pip install spacy

In [74]:
nlp = spacy.load("en_core_web_sm")

In [75]:
def tokenize_text(text):
    doc = nlp(text)
    return [token.text for token in doc]

In [76]:
df['tokens_spacy'] = df['remove_emoji'].apply(tokenize_text)

In [77]:
df['tokens_spacy']

0                 [Thesmiletextmisledwords, .]
1     [Heexperiencingtechnicaldifficulties, .]
2    [Spellcheckhelpcorrectmisspelledwords, .]
Name: tokens_spacy, dtype: object

In [79]:
nltk.download("all")

[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/abc.zip.
[nltk_data]    | Downloading package alpino to /root/nltk_data...
[nltk_data]    |   Unzipping corpora/alpino.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping
[nltk_data]    |       taggers/averaged_perceptron_tagger_ru.zip.
[nltk_data]    | Downloading package basque_grammars to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   Unzipping grammars/basque_grammars.zip.
[nltk_data]    | Downloading package bcp47 to /root/nltk_data...
[nltk_data]    | Downloading package biocreative_ppi to
[nltk_data]    |     /root/nltk_data...
[nltk_data]    |   U

True

In [84]:
from nltk.stem import WordNetLemmatizer
import nltk
nltk.download('wordnet')

# Load the English language model in spaCy
nlp = spacy.load("en_core_web_sm")

# Initialize the WordNet lemmatizer
lemmatizer = WordNetLemmatizer()
def lemmatization(tokens):
    lemmatized_words = [lemmatizer.lemmatize(word) for word in tokens]
    return ' '.join(lemmatized_words)

[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [85]:
df['lemmatization'] = df['tokens_spacy'].apply(lemmatization)

In [86]:
df["lemmatization"]

0                 Thesmiletextmisledwords .
1     Heexperiencingtechnicaldifficulties .
2    Spellcheckhelpcorrectmisspelledwords .
Name: lemmatization, dtype: object